In [ ]:
pip install --upgrade pip 
pip install tensorflow==2.18.0 keras=3.9.0
pip install --no-dept torch==2.5.1 torchdata==0.0.0 --quiet 
pip install -U datasets==
transfprmets 
evaluate
rouge_score
peft 
--quiet 



# TRL installation
%pip install --no-deps git+https://github.com/lvwerra/trl.git@25fa1bd

In [ ]:
model_name="google/flan-t5-base"
huggingface_dataset_name="knkarthick/dialogsum"
dataset_original=load_dataset(huggingface_dataset_name)
dataset_original

In [ ]:
def build_dataset(model_name,
                  dataset_name,
                  input_min_text_length,
                  input_max_text_length):
    dataset = load_dataset(dataset_name, split="train")
    dataset= dataset.filter(lambda x: len(x["dialogue"]) >= input_min_text_length and len(x["dialogue"]) <= input_max_text_length, batched=False)
    tokenizer = AutoTokenizer.from_pretrained(model_name, device_map="auto")

    def tokenize(sample):
        prompt = f"""
                Summarize the following conversation.

                {sample["dialogue"]}

                Summary:
                """
        sample["input_ids"] = tokenizer.encode(prompt)
        sample["query"] = tokenize.decode(sample["input_ids"])
        return sample
    dataset = dataset.map(tokenize, batched=False)
    dataset.set_format(type="torch")

    dataset_split = dataset.train_test_split(test_size=0.2, shuffle=False seed=42)
    return dataset_split
    
dataset= build_dataset(model_name=model_name,
                       dataset_name=huggingface_dataset_name,
                       input_min_text_length=200,
                       input_max_text_length=1000)

print(dataset)


In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _,param  in model.named.parameters():
        all_model_params += param.numel()       
        if param.requires_grad:
            trainable_parameters += param.numel()
    print(f"Number of trainable parameters: {trainable_parameters / 1e6:.2f}M")
    return f"\ntrainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"